In [2]:
!pip install feedparser # 설치 : RSS에서 xml태그별 정보를 추출(예: title, link, ...)
!pip install newspaper3k # 설치 : 인터넷신문기사분석(Article()을 사용하기 위한 목적) 파이썬 3버전이라 3k, 링크를 기사 들어가서 분석
!pip install konlpy # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install soynlp # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install lxml[html_clean] # 신규추가(2024년 가을)
import feedparser # !pip install feedparser
from newspaper import Article # !pip install newspaper3k
from konlpy.tag import Okt # !pip install konlpy
from collections import Counter # 본문에 몇번이나 명사가 나오는지 확인(TF구현용)
from bs4 import BeautifulSoup # 글에 존재할지 모르는 단어 추출

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=f4b735ff93ac95eb94343f68953a1d88f6be64680d9b505584bf643e36ea55c7
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=2c6f5c74b675dd61cd799717134f5d06489089f38b59032a35f856af69a95372
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df4

In [3]:
# [단계1] 모든 RSS파일(xml형식)을 돌아다니면서 기사의 제목/link를 추출
# urls는 우리가 검색할 RSS파일들의 목록을 list로 만든 것
urls = ["https://rss.nytimes.com/services/xml/rss/nyt/US.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Europe.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/World.xml"]
# 아래의 함수는 RSS목록의 list인 urls를 받아서 그 list에 존재하는
# 모든 RSS의 안에 있는 모든 기사들의 title과 link를 추출함
def crawl_rss(urls):
  array_rss = [] # 함수시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사의 title, link를 넣을 것임
  titles_rss = set() # [중복기사제거] 기사제목들의 집합을 구성함(집합은 중복을 허용하지 않기 때문임)
  for index_url in urls: # urls 리스트안에 하나씩 xml을 방문 (4번 방문예정 : 901, 902, 903, 904)
    print("[Crawl RSS]", index_url) # 현재 어떤 xml파일을 방문중인지 표시
    parse_rss = feedparser.parse(index_url) # 현재 xml파일을 파싱 후, 결과를 parse_rss에 저장
    for p in parse_rss.entries: # parse_rss에 있는 모든 entries/기사를 검색하면서
      if p.title not in titles_rss : # [중복기사제거] 기사제목집합에 현재 title이 업슬 때만 추가
        array_rss.append({'title':p.title,'link':p.link}) # append함수로 array_rss에 title과 link를 붙여넣음
        titles_rss.add(p.title) # [중복기사제거] 집합에 기사제목을 추가
      else:
        print("Duplicated Article:", p.title) # [중복기사제거] 중복되는 기사의 제목을 출력
  return array_rss
list_articles = crawl_rss(urls)
print(list_articles)

[Crawl RSS] https://rss.nytimes.com/services/xml/rss/nyt/US.xml
[Crawl RSS] https://rss.nytimes.com/services/xml/rss/nyt/Europe.xml
Duplicated Article: Republicans Make Early Inroads in Their Fight to Keep the House Majority
[Crawl RSS] https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml
Duplicated Article: Elon Musk Helped Elect Trump. What Does He Expect in Return?
[Crawl RSS] https://rss.nytimes.com/services/xml/rss/nyt/World.xml
Duplicated Article: World Reacts to Donald Trump’s Win in U.S. Presidential Election
Duplicated Article: For Ukraine, Trump Victory Signals a Shift. To What, Is Unclear.
Duplicated Article: Germany’s Coalition Collapses, Leaving the Government Teetering
Duplicated Article: How Kemi Badenoch Became Leader of the UK Conservative Party
Duplicated Article: Russian Plot Placed Incendiary Devices on Cargo Planes in Europe, Officials Say
Duplicated Article: North Korea Enters Ukraine Fight for First Time, Officials Say
Duplicated Article: Thursday Briefing

In [9]:
# [단계2] list에 존재하는 모든 기사드릐 link를 하나씩 돌아다니며 본문 title과 text 추출
# 아래의 함수는 하나의 url을 입력받아서, 그 링크를 타고 들어가, 그 안에 title과 text를 추출함.
def crawl_article(url, language='ko'):
  print("[Crawl Article]", url) # 현재 text를 추출할 기사의 url 출력
  article = Article(url, language=language) # Article을 사용하여 그 url을 입력하고, 언어옵션지정
  article.download() # 해당하는 url기사 다운로드
  article.parse() # 해당하는 url기사 분석
  return article.title, article.text # 해당하는 url기사 title과 text 출력

for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  # _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article함수에 넣어 본문 추출
  _, text = crawl_article(article['link'] , 'en') # 영어신문기사일 겨우의 옵션 적용
  article['text'] = text # 그 추출한 본문을 list_articles에 'text'라는 속성으로 새로 만들어 저장
print(list_articles[0])

[Crawl Article] https://www.nytimes.com/2024/11/06/us/running-ultramarathon-autism.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/trump-immigration-border.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/politics/harris-voters-democracy.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/malibu-fire-power-outages-california.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/dearborn-michigan-trump-arab-voters.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/politics/trump-president-agenda.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/jan-6-rioter-trump-pardon-judge-denies.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/politics/donald-trump-2024-campaign-coalition.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/politics/trump-election-covid-2020.html
[Crawl Article] https://www.nytimes.com/2024/11/06/us/politics/montgomery-county-pa-absentee-ballots.html
[Crawl Article] https://www.nytimes.com/2024/11/06/

In [10]:
import nltk
import spacy
nltk.download('punkt') # 토큰
nltk.download('webtext') # 텍스트 모음
nltk.download('stopwords') # 불용어

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
list_keywords = []
from nltk.tokenize import word_tokenize # 단어토큰화
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords # 불용어 가져오는 패키지
sp = list_articles[0]['text']

# tokenizer = RegexpTokenizer('[\w"]+') # 패턴, 불용어 등 분석 의미x 제거
# tokens = tokenizer.tokenize(sp.lower())
# print(tokens)

# en_stops = set(stopwords.words('english')) # 불용어 패키지
# res = [word for word in tokens if word not in en_stops] # 불용어 제외한 단어 추출
# print(res)

# import spacy

# nlp = spacy.load("en_core_web_sm")

# en_stops = set(stopwords.words('english'))

# # spacy 문서로 텍스트 처리
# doc = nlp(sp)

# 표제어 추출 및 불용어 제외
# lemmas = [token.lemma_ for token in doc if token.lemma_ not in en_stops and not token.is_punct]
# lemmas

# 불용어 리스트 (nltk에서 가져온 불용어 리스트)
nlp = spacy.load("en_core_web_sm")
en_stops = set(stopwords.words('english'))

# RegexpTokenizer로 텍스트에서 알파벳과 숫자만 추출 (구두점 제외)
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(sp.lower())  # 소문자로 변환하여 토큰화

# 불용어 제외한 단어들
filtered_tokens = [word for word in tokens if word not in en_stops]

# spacy를 이용하여 표제어 추출
doc = nlp(" ".join(filtered_tokens))  # 필터링된 토큰을 다시 하나의 문자열로 결합
lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
print("Lemmas:", lemmas)

Lemmas: ['cocodona', '250', 'say', 'banner', 'start', 'line', 'adventure', 'await', 'course', 'well', 'mark', 'zach', '22', 'digital', 'tracking', 'device', 'zach', 'mother', 'rana', 'worried', 'son', 'autism', 'prone', 'lose', 'way', 'rule', 'prevent', 'use', 'official', 'pace', 'runner', 'first', '78', 'mile', 'race', 'rana', 'find', 'another', 'competitor', 'willing', 'run', 'alongside', 'zach', 'informally', 'find', 'rana', 'stand', 'tiptoe', 'search', 'runner', 'sea', '278', 'participant', 'meticulously', 'plan', 'every', 'mile', 'zach', 'race', 'clock', 'tick', '5', 'start', 'voice', 'reveal', 'agitation']


In [14]:
# [단계3] 긁어온 본문 text에서 명사추출(키워드, 빈도수)
def get_keywords(text, nKeywords=10):
    list_keywords = []
    nlp = spacy.load("en_core_web_sm")
    en_stops = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())  # text를 소문자로 변환하여 토큰화
    filtered_tokens = [word for word in tokens if word not in en_stops]
    doc = nlp(" ".join(filtered_tokens))
    nouns = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and token.pos_ == "NOUN"]
    count = Counter(nouns)
    for n, c in count.most_common(nKeywords):
        item = {'keyword': n, 'count': c}
        list_keywords.append(item)
    return list_keywords

get_keywords(article['text'])
for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  keywords = get_keywords(article['text'], nKeywords=10) # 그 기사의 text를 get_keywords함수에 넣어 키워드/빈도 추출
  article['keywords'] = keywords # 그 추출한 키워드와 빈도수를 list_articles에 저장
print(list_articles[:5])

[{'title': 'When Running 250 Miles Is the Easy Part', 'link': 'https://www.nytimes.com/2024/11/06/us/running-ultramarathon-autism.html', 'text': '“Cocodona 250,” said the banner at the starting line. “Adventure awaits.”\n\nThe course was well-marked, and Zach, 22, had a digital tracking device. But Zach’s mother, Rana, was worried. Her son has autism and is prone to losing his way. The rules prevented him from using an official pace runner during the first 78 miles of the race, so Rana had found another competitor who was willing to run alongside Zach informally.\n\nBut now she couldn’t find him.\n\nRana stood on her tiptoes, searching for the runner in a sea of 278 participants. She had meticulously planned every mile of Zach’s race, and, as the clock ticked down to the 5 a.m. start, her voice revealed her agitation.', 'keywords': [{'keyword': 'zach', 'count': 3}, {'keyword': 'mile', 'count': 2}, {'keyword': 'race', 'count': 2}, {'keyword': 'banner', 'count': 1}, {'keyword': 'line', '

In [18]:
# [단계4] 검색어를 입력받아서 그 검색어를 가지고 있는 문서를 출력
query = input("질의어를 입력하세요 : ") # 쿼리를 입력 받음

# 아래의 함수는 쿼리를 입력받은 후에, 정해진 문서의 keywords의 리스트에서
# 쿼리가 그 keyword 중의 하나로 존재하는지 검색
def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 빈도수 출력(없으면 0)
  nWords = 0 # 아래의 if문에 걸리지 않으면(즉, 쿼리가 키워드에 없으면) 0을 출력하기 위한 초기값
  for kw in list_keywords: # 키워드 리스트를 하나씩 kw로 검색
    if query == kw['keyword']: # 만약에 쿼리와 동일한 키워드가 존재한다면
      nWords = kw['count'] # 그렇다면, 그 키워드에 해당하는 count가 nWords가 됨
  return nWords # 결과적으로 쿼리의 출현횟수가 출력됨


sorted_query = [] # 정렬된 기사 저장

for article in list_articles: # 기존에 만든 list_articles에서 하나의 기사를 방문하면서
  TF = search_articles(query, article['keywords']) # search_articles함수로 쿼리의 빈도수 추출
  if TF != 0: # 만약에 쿼리의 빈도수가 0이 아니면, 쿼리를 키워드로 가지고 있는 문서이므로, 관련 정보 출력
    sorted_query.append([TF, article['title'],article['link']]) # TF와 article을 튜플로 저장

# (아래) 리스트를 정렬하는데,
# 역순으로 TF값이 큰것부터 나오게하고,
# key값을 x[0]으로하여 리스트속서으이 가장 앞에 있는 TF를 기준으로 정렬
sorted_query.sort(reverse=True, key=lambda x:x[0])
print('-----------------------')
for article in sorted_query:
    print('[TF]',article[0])
    print(article[1])
    print(article[2])
    print('-----------------------')

질의어를 입력하세요 : trump
-----------------------
[TF] 17
Netanyahu Faces Backlash for Firing Israel’s Defense Minister
https://www.nytimes.com/live/2024/11/06/world/israel-gaza-iran-lebanon
-----------------------
[TF] 4
Elon Musk Helped Elect Trump. What Does He Expect in Return?
https://www.nytimes.com/2024/11/06/us/politics/elon-musk-trump-benefits.html
-----------------------
[TF] 4
Mitch McConnell and Other Top Republicans Upbeat After Trump’s Victory
https://www.nytimes.com/2024/11/06/us/politics/trump-republicans-cheney-mcconnell-romney.html
-----------------------
[TF] 4
Election Betting Markets See Vindication in Trump Victory
https://www.nytimes.com/2024/11/06/business/dealbook/prediction-markets-trump-win.html
-----------------------
[TF] 4
Bitcoin Hits a Record as Crypto Investors Root for a Trump Win
https://www.nytimes.com/2024/11/05/technology/bitcoin-election-trump.html
-----------------------
[TF] 4
Israeli Officials Embrace Trump Victory, Despite His Unpredictability
https: